In [46]:
#PIL、SKLearnを使ってデータを変換する
from PIL import Image
import os, time, sys

import keras
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.utils import np_utils

classes = ["monkey", "boar", "crow"]
num_classes = len(classes)
image_size = 50

My_decoder_name = "Animal_Decoder_aug"
My_data_name = "npydata01_aug"
My_input_shape = (50, 50, 3)
My_predict_image = "boar (2).jpg"

In [47]:
#予測に使うモデル → 学習に使ったモデルと全く同じ定義でOK。入力するテンソルのみ引数を変える
def build_model():
        model = Sequential()
        
        #一番最初の入力層には、モデルの画像サイズを使う
        #全結合をした後ドロップアウトすることで若干精度が向上する
        model.add(Conv2D(filters=32, kernel_size=(3,3), padding="same", input_shape=My_input_shape))
        model.add(Activation("relu"))
        model.add(Conv2D(filters=32, kernel_size=(3,3)))
        model.add(Activation("relu"))
        model.add(MaxPooling2D(pool_size=(2,2)))
        model.add(Dropout(0.25))
    
        model.add(Conv2D(filters=64, kernel_size=(3,3),padding="same", input_shape=My_input_shape))
        model.add(Activation("relu"))
        model.add(Conv2D(filters=64, kernel_size=(3,3)))
        model.add(Activation("relu"))
        model.add(MaxPooling2D(pool_size=(2,2)))
        model.add(Dropout(0.25))
    
        #全結合層に入力するため、Flattenレイヤーで4階テンソルとなっている出力値を2階テンソルに直す          
        model.add(Flatten())
    
        #全結合層
        model.add(Dense(units=512))
        model.add(Activation("relu"))
        model.add(Dropout(0.5))
        model.add(Dense(units=3))
        model.add(Activation("softmax"))
        model.output_shape
    
        #モデルをコンパイル + 損失関数・最適化関数を定義
        model.compile(loss = "categorical_crossentropy", optimizer="adam", metrics = ["accuracy"])
    
        #学習したモデルをロードする
        model = load_model("./" + My_decoder_name + ".h5")

        return model

#メイン関数 - 任意のイメージがどの物体化を予測する
def main():
    image = Image.open(My_predict_image)
    image = image.convert("RGB")
    image = image.resize((image_size,image_size))
    data = np.asarray(image)

    X = []
    X.append(data)
    X = np.array(X)
    model = build_model()

    result = model.predict([X])[0]
    predicted = result.argmax()
    percentage = result[predicted] * 100
    print(result)
    print("Object is {0}, with {1} probability.".format(classes[predicted], percentage))

In [48]:
if __name__ == "__main__":
    main()

1/1 [==============================] - 0s 84ms/step
[0. 1. 0.]
Object is boar, with 100.0 probability.
